In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
import numpy as np

### Generar los datos para entrenar el modelo

In [2]:
operator_encoding = {'+': 0, '-': 1, '*': 2, '/': 3}

def generate_all_data():
    X_op = []
    X_num1 = []
    X_num2 = []
    y = []
    
    for op in operator_encoding.keys():
        for num1 in range(10):
            for num2 in range(10):
                if op == '/' and num2 == 0:
                    continue
                
                op_encoded = operator_encoding[op]
                
                if op == '+':
                    res = num1 + num2
                elif op == '-':
                    res = num1 - num2
                elif op == '*':
                    res = num1 * num2
                elif op == '/':
                    res = num1 / num2
                
                X_op.append([op_encoded])
                X_num1.append([num1])
                X_num2.append([num2])
                y.append([res])
    
    return [np.array(X_op), np.array(X_num1), np.array(X_num2)], np.array(y)

### Crear el modelo

In [3]:
def create_model():
    op_input = Input(shape=(1,), name='op_input')
    num1_input = Input(shape=(1,), name='num1_input')
    num2_input = Input(shape=(1,), name='num2_input')
    
    merged = Concatenate()([op_input, num1_input, num2_input])
    
    x = Dense(256, activation='relu')(merged)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    
    output = Dense(1, activation='linear')(x)
    
    model = Model(inputs=[op_input, num1_input, num2_input], outputs=output)
    
    model.compile(optimizer=Adam(0.0001),
                 loss='mse',
                 metrics=['mae'])
    
    return model

### Entrenamiento del modelo

In [7]:
X, y = generate_all_data()
model = create_model()
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.fit(X, y, epochs=600, batch_size=32, callbacks=[early_stop])

Epoch 1/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 188.2113 - mae: 7.7553  
Epoch 2/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 200.7081 - mae: 7.9679 
Epoch 3/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 178.6179 - mae: 7.5869 
Epoch 4/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 180.2645 - mae: 7.4334 
Epoch 5/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 146.6875 - mae: 7.4408
Epoch 6/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 151.7534 - mae: 7.5528 
Epoch 7/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 132.5341 - mae: 6.8663
Epoch 8/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 124.1443 - mae: 7.1370
Epoch 9/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 155.4522 - mae: 7.5675 
Epoch 10/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 182.7066 - mae: 8.2514 
Epoch 11/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 139.5060 - mae: 7.6361
Epoch 12/600
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 148.0810 - mae: 7.6264 
Epoc

### Funcion para probar el modelo

In [8]:
def predict(op, num1, num2, verbose = False):
    op_encoded = np.array([[operator_encoding[op]]])
    num1_arr = np.array([[num1]])
    num2_arr = np.array([[num2]])
    
    pred = model.predict([op_encoded, num1_arr, num2_arr])[0][0]
    
    if op in ['+', '-', '*']:
        pred = round(pred)
    
    if(verbose):
        if op == '+':
            real_value = num1 + num2
        elif op == '-':
            real_value = num1 - num2
        elif op == '*':
            real_value = num1 * num2
        elif op == '/':
            real_value = num1 / num2 if num2 != 0 else float('inf')
        else:
            raise ValueError(f"Operador desconocido: {op}")

        error = abs(pred - real_value)
        print(f"{num1} {op} {num2} = Pred: {pred} | Real: {real_value} | Error: {error}")
    return pred

### Realizar predicciones

In [10]:
predict("+", 1, 2, True)
predict("-", 9, 3, True)
predict("*", 2, 4, True)
predict("/", 4, 2, True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1 + 2 = Pred: 3 | Real: 3 | Error: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
9 - 3 = Pred: 7 | Real: 6 | Error: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2 * 4 = Pred: 8 | Real: 8 | Error: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4 / 2 = Pred: 2.1075968742370605 | Real: 2.0 | Error: 0.10759687423706055


np.float32(2.1075969)